# Mutual Unbiased Bases as a Countermeasure to Barren Plateaus in Variational Quantum Algorithms

### Imports and Globals

In [10]:
import qiskit as qk
from qiskit import Aer, QuantumCircuit

from qiskit.circuit import Parameter, ClassicalRegister
from qiskit.circuit.library import EfficientSU2
from qiskit.utils import QuantumInstance
from qiskit.algorithms import VQE
from qiskit.algorithms.minimum_eigen_solvers.vqe import VQEResult
from qiskit.algorithms.optimizers import COBYLA
from typing import Tuple, List, Dict, Union
from scipy.optimize import minimize, OptimizeResult
import numpy as np
from random import random
import json
from pprint import pprint
import matplotlib.pyplot as plt
from operator import truediv
from cmath import pi
from random import randint
import os
import time
%matplotlib inline

# MUB circuit file paths
MUB_CIRC_2_PATH = os.path.join(os.getcwd(), 'mub_bqskit', '2_302')
MUB_CIRC_3_PATH = os.path.join(os.getcwd(), 'mub_bqskit', '3_306')

# Result folder names paths
VQC_FOLDER = 'VQC results'
NO_MUB_GRAPH_FOLDER = '3 qubit graphs'
N_QUBIT_NO_MUB_GRAPH_FOLDER = 'n qubit graphs'

# Hyper-parameter value options
EPS_TOL = 1e-12
LO_TOL = 1e-5
MID_TOL = 1e-2
HI_TOL = 0.2
TINY_SUCCESS_BOUND = 0.01
LO_SUCCESS_BOUND = 0.1
HI_SUCCESS_BOUND = 0.4

# Maximal Iteration upper bound (set to not be reached)
MAX_ITER = 1e8

# Circuit Execution Parameters
SHOTS = 8192
qasm_backend = Aer.get_backend('qasm_simulator')
qasm_qi = QuantumInstance(qasm_backend, shots=SHOTS)

### Basic Barren Plateau Circuit for Variational Quantum Compilation

The "Arrasmith" ansatz circuit is taken from "Effect of barren plateaus on gradient-free optimization" by Arrasmith et al. (2021).

The attempted task is "trivial" Variational Quantum Compilation.
Vartational Quantum Compilation gets some unitary $U$, and an ansatz $V(\theta)$, and attempts to find a value for $\theta$ such that $V(\theta)| 0 \rangle = U | 0 \rangle$.

In this case, we choose $U=I$.
Because we pick a *random* initial guess for $\theta$, we will experience the barren plateaus that occur when the $\theta$ values are away from the target.
I took this specific problem from "Effect of barren plateaus on gradient-free optimization" by Arrasmith et al. (2021).

Note, however, an important observation:
In order to actually use the "value" of the different MUB starting points, the original value of the parameters needs to be constant (although random) for all experiments.

In [11]:
def gen_vqc_ansatz_arrasmith(n_qubits: int, n_layers: int) -> QuantumCircuit:
    qc = qk.QuantumCircuit(n_qubits)
    count=1
    for _ in range(n_layers):
        
        # First layer of U3 gates
        for i in range(n_qubits):
            thetas = [Parameter(f'theta_{count+z}') for z in range(3)]
            qc.u(*thetas, i)
            count += 3
            
        # First layer of even CXs
        for i in range(0, n_qubits-1, 2):
            qc.cx(i, i+1)
            
        # Second layer of U3 gates
        for i in range(n_qubits):
            thetas = [Parameter(f'theta_{count+z}') for z in range(3)]
            qc.u(*thetas, i)
            count += 3
            
        for i in range(1, n_qubits, 2):
            qc.cx(i, (i+1)%n_qubits)
            
    qc.measure_all()
        
    return qc

### Default Experiment Parameters and Hyper-parameters

In [12]:
optimizer = ''
success_bound = tol = K = N_LAYERS = N_QUBITS= 0
def std_exp_3_qubits():
    global optimizer
    global success_bound
    global tol
    global K
    global N_LAYERS
    global N_QUBITS
    optimizer = 'COBYLA'
    success_bound = LO_SUCCESS_BOUND
    tol = LO_TOL
    K = 25
    N_LAYERS = 12
    N_QUBITS = 3
    
def std_exp_7_qubits():
    global optimizer
    global success_bound
    global tol
    global K
    global N_LAYERS
    global N_QUBITS
    optimizer = 'COBYLA'
    success_bound = LO_SUCCESS_BOUND
    tol = LO_TOL
    K = 25
    N_LAYERS = 7
    N_QUBITS = 7

std_exp_3_qubits()

### Experiment Functions - Generic VQC experiment

In [13]:
def low_constraint(x: List[float]):
    return min(x)

def hi_constraint(x: List[float]):
    return 2*pi-max(x)

def get_constraints() -> List[dict]:
    return [{'type': 'ineq', 'fun': low_constraint}, {'type': 'ineq', 'fun': hi_constraint}]

def rand_angle():
    return (np.random.random()) * 2 * pi - pi

# Calculate the percentage of results where 0 was reached in the jth qubit.
def get_p0j(counts: dict, j: int, n_qubits: int, shots: int = SHOTS) -> float:
    assert j >= 0 and j < n_qubits
    count0j = sum([v for k,v in counts.items() if k[j] == '0'])
    assert count0j >= 0 and count0j <= shots
    return count0j / shots

# Returns the number of function evaluations it took for the method to converge.
# If during the optimization, the cost function goes below a specified bound, the optimization halts.
# If the optimization halted because of the bound, the OptimizeResult is replaced with None.
def run_vqc_exp(qc: QuantumCircuit,
                theta0: List[float],
                tol: float = HI_TOL,
                success_bound: float = 0,
                optimizer: str = 'COBYLA',
                qi: QuantumInstance = qasm_qi,
                q: float = 0) -> Tuple[List[float], OptimizeResult]:
    
    # defining 
    n_qubits = qc.num_qubits
    cost_points = []
    
    # Exception used for breaking on hitting success bound
    class BoundHitException(Exception):
        pass
    
    
    def evaluate_cost(theta: List[float]) -> float:
        if float('inf') in theta or float('-inf') in theta:
            raise ZeroDivisionError()
        concrete_qc = qc.bind_parameters(theta)
        results = qi.execute(concrete_qc)
        counts = dict(results.get_counts())
        # Calculating the cost
        # Note that the Arrasmith paper uses ONLY the local cost.
        # Also, note that while the initial global cost is close to 1, the initial local cost should be around 0.5.
        # The intuition is that the probability to be in |0> specifically vanishes exponentially,
        # While the amount of "lit" qubits in a random answer should be around half.
        global_cost = 1 - (results.get_counts().int_raw.get(0, 0) / SHOTS)
        local_cost = 1 - sum([get_p0j(counts, j, n_qubits) for j in range(n_qubits)]) / n_qubits
        cost = q*global_cost + (1-q)*local_cost
        cost_points.append(cost)
        if (cost) <= success_bound:
            print('Success bound reached. WOOP')
            raise BoundHitException()
        return cost

    try:
        # Based on constraint functions
        if optimizer == 'COBYLA':
            res = minimize(evaluate_cost,
                theta0,
                method='COBYLA',
                options={'maxiter': MAX_ITER},
                tol=tol,
                constraints=get_constraints())
        #based on concrete bounds
        elif optimizer in ['Nelder-Mead', 'SLSQP', 'Powell']:
            res = minimize(evaluate_cost,
                theta0,
                method=optimizer,
                options={'maxiter': MAX_ITER},
                tol=tol,
                bounds = [(-pi,pi) for _ in range(len(theta0))])
        else:
            assert 0==1
        
    except BoundHitException:
        return cost_points, True
    except ZeroDivisionError:
        return cost_points, cost_grad
    except KeyboardInterrupt:
        print('Optimization stopped by keyboard interrupt. WOLOLO')
        return cost_points, False

    return cost_points, res

### Experiment Functions - prepending circuits

In [14]:
# This function gets a number i from 0 to (2^n)-1
# and returns a circuit that generates the state |i> when acting on |0>.
def get_comp_state_circ(state_idx: int, n_qubits: int) -> QuantumCircuit:
    qc = QuantumCircuit(n_qubits)
    bin_str = bin(state_idx)[2:].zfill(n_qubits)
    for i, ch in enumerate(bin_str):
        if ch == '1':
            qc.x(i)
    return qc

comp_circuits = [get_comp_state_circ(i, N_QUBITS) for i in range(2 ** N_QUBITS)]


def run_vqc_exp_with_state_prepend(ansatz_qc: QuantumCircuit, state_qc: QuantumCircuit,
                                   n_qubits: int, theta0: List[float],
                                   tol: float = 0.2, success_bound: float=0,
                                   optimizer: str = 'COBYLA',
                                   qi: QuantumInstance = qasm_qi) -> dict:
    state_qc = state_qc.copy()
    state_qc.add_register(ClassicalRegister(n_qubits))
    full_qc = state_qc.compose(ansatz_qc, qubits=range(n_qubits), inplace=False)
    assert full_qc != None
    return run_vqc_exp(full_qc, theta0, tol, success_bound, optimizer, qi)


def run_vqc_exp_with_mub_prepend(ansatz_qc: QuantumCircuit, mub_qc: QuantumCircuit,
                                 n_qubits: int, n_layers: int,
                                 theta0: List[float], tol: float = 0.2,
                                 success_bound: float = 0,
                                 optimizer: str = 'COBYLA',
                                 qi: QuantumInstance = qasm_qi) -> dict:
    mub_qc = mub_qc.copy()
    mub_qc.add_register(ClassicalRegister(n_qubits))
    mub_ansatz_qc = mub_qc.compose(ansatz_qc, qubits=range(n_qubits), inplace=False)
    assert mub_ansatz_qc != None

    res_dict = {}
    for i in range((2 ** n_qubits)):
        state_qc = get_comp_state_circ(i, n_qubits)
        res_dict[i] = run_vqc_exp_with_state_prepend(mub_ansatz_qc, state_qc, n_qubits, theta0, tol, success_bound, optimizer, qi)
    return res_dict


In [15]:
std_exp_3_qubits()

### Control Group: $k$ Random Initial Parameter Vectors

#### Result Data Structure

The results are a list of tuples.
The first element is the theta vector used for that run.
The second element is a tuple of the convergence graph, and (optionally) the result data structure generated by SciPy.

In [20]:
def run_k_random_theta_vqc_exp_set():
    print('=====Experimenting with SMALL SETS of radnom initial parameters=====')
    start_time = time.time()
    first_time = start_time
    finish_time = start_time
    interrupted = False
    
    results = []
    ansatz = gen_vqc_ansatz_arrasmith(N_QUBITS, N_LAYERS)

    for i in range(K):
        theta0 = [rand_angle() for _ in range(ansatz.num_parameters)]
        print(f'experimenting with random parameter set #{i}')
        exp_data = run_vqc_exp(ansatz, theta0, tol=tol, success_bound=success_bound, optimizer=optimizer)
        print(f'parameter set #{i} done')
        
        if exp_data[1] == False: # Signifying keyboard interrupt
            interrupted = True
            exp_data = (exp_data[0], [])
        elif exp_data[1] == True:
            state_data = (exp_data[0], [])

        finish_time = time.time()
        print(f'Finished experiment in {finish_time - start_time} seconds at {time.asctime(time.localtime())}')
        start_time = finish_time

        results.append((theta0, exp_data))

        if interrupted:
            return results, False
        
    print(f'total took {finish_time - first_time} seconds')
    
    return results, True

### Loading the QASM MUB circuits into Qiskit

In [60]:
CIRC_FROM_FILES = True
if CIRC_FROM_FILES:
    paths_2 = os.listdir(MUB_CIRC_2_PATH)
    paths_3 = os.listdir(MUB_CIRC_3_PATH)
    mub_circuits_2_qubits = [qk.circuit.QuantumCircuit.from_qasm_file(os.path.join(MUB_CIRC_2_PATH,path)) for path in paths_2 if '.txt' in path]
    mub_circuits_3_qubits = [qk.circuit.QuantumCircuit.from_qasm_file(os.path.join(MUB_CIRC_3_PATH,path)) for path in paths_3 if '.txt' in path]
else:
    mub_circuits_2_qubits = [qk.circuit.QuantumCircuit.from_qasm_str(qasm_str) for qasm_str in qasm_2_302.values()]
    mub_circuits_3_qubits = [qk.circuit.QuantumCircuit.from_qasm_str(qasm_str) for qasm_str in qasm_3_306.values()]

### Utilization Method: try k random MUB states

#### Results Data Structure
The results of the experiment are a dictionary.
The key is a pair: the first element is the MUB index, and the second is the state index.
The value is a tuple of the convergence graph, and (optionally) the result data structure generated by SciPy.

In [66]:
def generate_k_mub_choices(k: int) -> List[Tuple[int, int]]:
    n_mubs = 2 ** N_QUBITS + 1
    n_states = 2 ** N_QUBITS
    assert k <= n_mubs * n_states
    res = []
    while k > 0:
        while True:
            mub = randint(0, n_mubs-1)
            state = randint(0, n_states-1)
            if (mub, state) not in res:
                res.append((mub, state))
                k -= 1
                break
    return res

In [22]:
def run_k_mub_state_vqc_exp_set():
    print('=====Experimenting with SMALL SETS of prepended MUB states=====')
    start_time = time.time()
    first_time = start_time
    finish_time = start_time
    interrupted = False
    results = {}
    
    ansatz = gen_vqc_ansatz_arrasmith(N_QUBITS, N_LAYERS)
    theta0 = [rand_angle() for _ in range(ansatz.num_parameters)]
    state_choices = generate_k_mub_choices(K)

    for mub, state in state_choices:
        print(f'experimenting with MUB {mub}, state {state}')
        mub_qc = mub_circuits_3_qubits[mub].copy()
        state_qc = comp_circuits[state].copy()
        full_state_qc = state_qc.compose(mub_qc, inplace=False)
        state_data = run_vqc_exp_with_state_prepend(ansatz, full_state_qc, N_QUBITS,
                                                    theta0, tol=tol, success_bound=success_bound, optimizer=optimizer)
        print(f'({mub},{state}) done')

        if state_data[1] == False: # Signifying keyboard interrupt
            interrupted = True
            state_data = (state_data[0], [])
        elif state_data[1] == True:
            state_data = (state_data[0], [])

        finish_time = time.time()
        print(f'Finished experiment in {finish_time - start_time} seconds at {time.asctime(time.localtime())}')
        start_time = finish_time

        results[(mub, state)] = state_data

        if interrupted:
            return results, False

    print(f'total took {finish_time - first_time} seconds')
    return results, True

In [ ]:
k_prepend_mub_results = run_k_mub_state_vqc_exp_set()

### Experiments: Try k half-MUBs

When working with $n$ qubits, there is an exponential number of MUBs, and each MUB has an exponential number of states.

The generation of all of these MUB transformations takes exponential time by itself.

To mitigate that, Tal proposed the following idea:
Pick a pair of qubits in an $n$-qubit register. generate a 2-qubit MUB state between them, and put the rest on 0.

This method has a polynomial amount of states, and spans an interesting part of the space.

#### Results Data Structure
The results of the experiment are a dict from the n of layers and qubits to the results for that n.

Each layer's results are also a dictionary.
The key is a 4-tuple: the two first places are the pairs of qubits on which the MUB state was generated, the third is the MUB index, and the fourth is the state index.
The value is a tuple of the convergence graph, and (optionally) the result data structure generated by SciPy.

In [74]:
def gen_half_mub_circ(n_qubits: int, qubit_1: int, qubit_2: int, mub_no: int, state_no: int, mub_circuits: List[QuantumCircuit]) -> QuantumCircuit:
    qc = QuantumCircuit(n_qubits)
    comp_state_qc = get_comp_state_circ(state_no, n_qubits=2)
    qc.compose(comp_state_qc, qubits=[qubit_1, qubit_2], inplace=True)
    qc.compose(mub_circuits[mub_no], qubits=[qubit_1, qubit_2], inplace=True)
    return qc

def gen_k_half_mub_choices(n_qubits: int, k: int) -> List[Tuple[int, int, int, int]]:
    n_mubs = 2 ** 2 + 1
    n_states = 2 ** 2
    # m mubs * (n choose 2) pairs of qubits
    assert k <= n_mubs * n_qubits * (n_qubits-1) // 2
    res = []
    while k > 0:
        while True:
            qubit_1 = randint(0, n_qubits-2)
            qubit_2 = randint(qubit_1+1, n_qubits-1)
            mub = randint(0, n_mubs-1)
            state = randint(0, n_states-1)
            if (mub, state) not in res:
                res.append((qubit_1, qubit_2, mub, state))
                k -= 1
                break
    return res

In [ ]:
def run_k_half_mub_state_vqc_exp_set():
    print('=====Experimenting with SMALL SETS of prepended half-MUB states=====')
    start_time = time.time()
    first_time = start_time
    finish_time = start_time
    interrupted = False
    results = {}
    
    ansatz = gen_vqc_ansatz_arrasmith(N_QUBITS, N_LAYERS)
    theta0 = [rand_angle() for _ in range(ansatz.num_parameters)]
    state_choices = gen_k_half_mub_choices(n_qubits=N_QUBITS, k=K)

    for choice in state_choices:
        print(f'experimenting with pair {(choice[0], choice[1])}, MUB {choice[2]}, state {choice[3]}')
        full_state_qc = gen_half_mub_circ(N_LAYERS, *choice, mub_circuits_2_qubits)
        state_data = run_vqc_exp_with_state_prepend(ansatz, full_state_qc, N_QUBITS,
                                                    theta0, tol=tol, success_bound=success_bound, optimizer=optimizer)
        print(f'{choice} done')
        
        finish_time = time.time()
        print(f'Finished experiment in {finish_time - start_time} seconds at {time.asctime(time.localtime())}')
        start_time = finish_time

        results[choice] = state_data

        if interrupted:
            return results, False
        

    print(f'total took {finish_time - first_time} seconds')
    return results, True

In [ ]:
k_prepend_half_mub_results = run_k_half_mub_state_vqc_exp_set()

### Hyper-Parameter Experiments: Optimizer

In [21]:
std_exp_7_qubits()
optimizer = 'COBYLA'
random_thetas_results = {}
for n in range(4,9):
    N_QUBITS = N_LAYERS = n
    random_thetas_results[n], graceful = run_k_random_theta_vqc_exp_set()
    if not graceful:
        break
std_exp_7_qubits()

=====Experimenting with SMALL SETS of radnom initial parameters=====
experimenting with random parameter set #0
Optimization stopped by keyboard interrupt. WOLOLO
parameter set #0 done
Finished experiment in 3.7633442878723145 seconds at Mon Oct  3 16:34:10 2022


capi_return is NULL
Call-back cb_calcfc_in__cobyla__user__routines failed.


### Analysis Functions - General

In [8]:
def avg(col):
    return sum(col) / len(col) if len(col) > 0 else None

def wavg(col, weights):
    assert len(col) == len(weights)
    return sum([v * w for v,w in zip(col, weights)])


def nfev(data: Tuple[List[float], Union[VQEResult, OptimizeResult]]) -> int:
    evals, record = data
    if evals != []:
        return len(evals)
    elif type(record) == VQEResult:
        return record.cost_function_evals
    else:
        assert type(record) == OptimizeResult
        return record.nfev
    
    
def fin_val(data: Tuple[List[float], Union[VQEResult, OptimizeResult]]) -> int:
    evals, record = data
    if evals != []:
        return evals[-1]
    elif type(record) == VQEResult:
        return record.eigenvalue
    else:
        return record.fun

def plot_and_save_convergence_graph(evals: List[float], target_vals: List[float], title: str, filename: str):
    plt.axis([0, len(evals), 0, 1])
    plt.plot(evals)
    plt.title(title)
    
    if target_vals != []:
        plt.plot([0, len(evals)-1], [target_vals[0], target_vals[0]], color='red')
    for fake_target in target_vals[1:]:
        plt.plot([0, len(evals)-1], [target_vals[0], target_vals[0]], color='purple')
        
    plt.savefig(filename)
    plt.show()

### Analysis Functions - Utilization Methods

In [71]:
def gen_statistics_random_thetas(random_thetas_results_dict: List[Tuple[List[float],Tuple[List[float], VQEResult]]],
                                 no_mub_stats: Dict[int, Dict[str, int]],
                                 target_vals: List[float] = [0],
                                 stats_folder: str = '',
                                 stats_filename: str = '') -> Dict[int, Dict[str, any]]:
    
    if stats_folder == '':
        stats_folder = input('enter path for results directory')
    if not os.path.isdir(stats_folder):
        os.mkdir(stats_folder)
    if stats_filename == '':
        stats_filename = input('enter filename for statistics file')
        
    assert target_vals != []
    success_bound = target_vals[0]    
    
    stats = {}
    min_nfev = min([nfev(exp_data) for _, exp_data in random_thetas_results])
    stats['min_nfev'] = min_nfev
    stats['avg_nfev'] = avg([nfev(exp_data) for _, exp_data in random_thetas_results])
    correct_thetas = [fin_val(exp_data) for _, exp_data in random_thetas_results if abs(fin_val(exp_data)-success_bound) < 1e-2]
    stats['correct_thetas_count'] = len(correct_thetas)
    stats['correct_addition_percent'] = len(correct_thetas) / len(random_thetas_results)
    stats['correct_avg_val'] = avg(correct_thetas)

    stats['evals'] = {k: v[1][0] for k,v in enumerate(random_thetas_results)}
    stats['nfevs'] = {k: len(v[1][0]) for k,v in enumerate(random_thetas_results)}
    stats['fin_vals'] = {k: fin_val(v[1]) for k,v in enumerate(random_thetas_results)}
        
    with open(os.path.join(stats_folder, stats_filename), 'w') as f:
        pprint(stats, f)
        
    for idx, evals in stats['evals'].items():
        plot_and_save_convergence_graph(evals=evals,
                        target_vals=target_vals,
                        title=f'Convergence graph with {N_LAYERS} layers, theta #{idx}',
                        filename=os.path.join(stats_folder, f'{N_LAYERS},{idx}.png'))
    
    return stats



# Analyze the performance of using MUB states as initial states for an ansatz
def gen_statistics_mubs(prepend_results_dict: Dict[Tuple[int, int], Tuple[List[float], VQEResult]],
                              no_mub_stats: Dict[int, Dict[str, int]],
                              target_vals: List[float] = [0],
                              stats_folder: str = '',
                              stats_filename: str = '') -> Dict[int, Dict[str, any]]:
    
    if stats_folder == '':
        stats_folder = input('enter path for results directory')
        if not os.path.isdir(stats_folder):
            os.mkdir(stats_folder)
    if stats_filename == '':
        stats_filename = input('enter filename for statistics file')

    stats = {}
    min_nfev = min([nfev(state_data) for state_data in prepend_results_dict.values()])
    stats['min_nfev'] = min_nfev
    stats['avg_nfev'] = avg([nfev(state_data) for state_data in prepend_results_dict.values()])


    assert target_vals != []
    success_bound = target_vals[0]
    # Experiments that terminated closer to the intended result
    correct_states = {state: fin_val(state_data) for state_idx, state_data in prepend_results_dict.items() if abs(fin_val(state_data)-success_bound) < 1e-2}        
    stats['correct_states'] = correct_states
    stats['correct_states_count'] = len(correct_states)
    stats['correct_addition_percent'] = len(correct_states) / len(prepend_results_dict)
    stats['correct_avg_val'] = avg(correct_states.values())

    stats['evals'] = {k: v[0] for k,v in prepend_results_dict.items()}
    stats['nfevs'] = {k: len(v[0]) for k,v in prepend_results_dict.items()}
    stats['fin_vals'] = {k: fin_val(v) for k,v in prepend_results_dict.items()}
        
    with open(os.path.join(stats_folder, stats_filename), 'w') as f:
        pprint(stats, f)
        
    for idx, data in prepend_results_dict.items():
        plot_and_save_convergence_graph(evals=data[0],
                            target_vals=target_vals,
                            title=f'Convergence graph with {N_LAYERS} layers, state {idx}',
                            filename=os.path.join(stats_folder, f'{N_LAYERS},{idx[0]},{idx[1]}.png'))
    return stats


def gen_statistics_half_mubs(half_mubs_results_dict: Dict[int, Dict[Tuple[int, int], Tuple[List[float], VQEResult]]],
                             no_mub_stats: Dict[int, Dict[str, int]],
                             target_vals: List[float] = [0],
                             stats_folder: str = '',
                             stats_filename: str = '') -> Dict[int, Dict[str, any]]:
    
    if stats_folder == '':
        stats_folder = input('enter path for results directory')
        if not os.path.isdir(stats_folder):
            os.mkdir(stats_folder)
    if stats_filename == '':
        stats_filename = input('enter filename for statistics file')

    stats = {}
    min_nfev = min([nfev(state_data) for state_data in half_mubs_results_dict.values()])
    stats['min_nfev'] = min_nfev
    stats['avg_nfev'] = avg([nfev(state_data) for state_data in half_mubs_results_dict.values()])

    assert target_vals != []
    success_bound = target_vals[0]
    # Experiments that terminated closer to the intended result
    correct_states = {choice: fin_val(state_data) for choice, state_data in half_mubs_results_dict.items() if abs(fin_val(state_data)-success_bound) < 1e-2}        
    stats['correct_states'] = correct_states
    stats['correct_states_count'] = len(correct_states)
    stats['correct_addition_percent'] = len(correct_states) / len(half_mubs_results_dict)
    stats['correct_avg_val'] = avg(correct_states.values())

    stats['evals'] = {k: v[0] for k,v in half_mubs_results_dict.items()}
    stats['nfevs'] = {k: len(v[0]) for k,v in half_mubs_results_dict.items()}
    stats['fin_vals'] = {k: fin_val(v) for k,v in half_mubs_results_dict.items()}
        
    with open(os.path.join(stats_folder, stats_filename), 'w') as f:
        pprint(stats, f)
        
    for idx, data in half_mubs_results_dict.items():
        plot_and_save_convergence_graph(evals=data[0],
                            target_vals=target_vals,
                            title=f'Convergence graph with {N_LAYERS} layers, pair ({idx[0]},{idx[1]}), MUB state({idx[2]},{idx[3]})',
                            filename=os.path.join(stats_folder, f'{N_LAYERS},{idx[0]},{idx[1]},{idx[2]},{idx[3]}.png'))
    return half_mub_stats

### Analysis

In [ ]:
prepending_stats = gen_statistics_mubs(k_prepend_mub_results,
                                             {},
                                             target_vals=[success_bound])

In [ ]:
k_random_stats = gen_statistics_random_thetas(random_thetas_results,
                                              {},
                                              target_vals=[success_bound],
                                             stats_folder='VQC results/local_cost_bps_3_qubits',
                                             stats_filename='stats.txt')

In [ ]:
half_mub_stats = gen_statistics_half_mubs(k_prepend_half_mub_results,
                                          {},
                                          target_vals=[success_bound],
                                         stats_folder='VQC results/BBB',
                                         stats_filename = 'bbb.txt')

## Collective Experiments - VQC

### Plotting Functions

In [137]:
def myplot(data: dict, label: str, filename: str = '', logscale=False):
    data = {k: v for k,v in data.items() if v != None}
    lists = sorted(data.items()) # sorted by key, return a list of tuples
    x, y = zip(*lists) # unpack a list of pairs into two tuples
    if logscale:
        plt.yscale('log')
        
    if filename == '':
        filename = label
    plt.savefig(filename)
    
    plt.plot(x, y, label=label, marker='o')
    

def myplot_heatmap(data: list, label:str, target_vals: list = [], val_range=(-0.1, 0.7), bin_count=50, filename=""):
    plt.hist(data, bin_count, range=val_range, color='blue')

    
    if target_vals != []:
        plt.plot([target_vals[0], target_vals[0]], [0,10], color='green', linewidth=1)
    for fake_target in target_vals[1:]:
        plt.plot([fake_target, fake_target], [0,10], color='red', linewidth=1)

    plt.title(label)
    
    if filename == '':
        filename = label
    plt.savefig(filename)
    
    plt.show()
    

### (Optionally) Load Data Fom Files

In [ ]:
from ast import literal_eval

STATS_FROM_FILES = True

def load_stats(name: str) -> dict:
    filename = os.path.join(os.getcwd(), VQC_FOLDER, name)
    with open(filename, 'r') as f:
        data = f.read()
    
    return literal_eval(data)

if STATS_FROM_FILES:
    no_mub_stats = load_stats(NO_MUB_PATH)
    prepend_stats = load_stats(PRE_MUB_PATH)

### Raw Summary Data

In [ ]:
print('No MUBs:')
print(no_mub_stats)
print('Prepended MUB states:')
print(prepend_stats)

In [ ]:
# plt.hist([layer['fin_val'] for layer in no_mub_stats.values()], 50, range=(-0.1, 0.5), density=True)
# plt.plot([0,0], [0,5])
# plt.title('final values without MUBS')
# plt.show()

# for layer, layer_stats in prepend_stats.items():
#     plt.hist(layer_stats['fin_vals'], 50, range=(-0.1, 0.5), density=True)

#     # Add Correct Result Line
#     plt.plot([0,0], [0,30], color='green', linewidth=1)
#     plt.title(f'function evaluations for {layer} layers')
#     plt.show()

myplot_heatmap([layer['fin_val'] for layer in no_mub_stats.values()], 0.0, title='final values without using MUBs', bin_count=50)
for layer, layer_stats in prepend_stats.items():
    myplot_heatmap(layer_stats['fin_vals'], 0.0, title=f'function evaluations for {layer} layers', bin_count=50)



### Minimal \# Of Function Evals

In [ ]:
def plot_min_nfev(stats_dict: Dict[int, Dict[str, any]], label: str):
    for v in stats_dict.values():
        assert 'min_nfev' in v.keys() or 'nfev' in v.keys()
    nfev_dict = {k: v['min_nfev'] if 'min_nfev' in v.keys() else v['nfev'] for k,v in stats_dict.items()}
    myplot(nfev_dict, label)

plt.xlabel("# Layers")
plt.ylabel("Function Evals")
plt.title('Minimal # of Function Evals per layer')
plot_min_nfev(no_mub_stats, "No MUBs")
plot_min_nfev(prepend_stats, "Prepended MUB states")
plot_min_nfev(append_stats, "Appended MUB trans.")
plt.legend()
plt.show()

### Average \# Of Function Evals

In [ ]:
def plot_avg_nfev(stats_dict: Dict[int, Dict[str, any]], label: str):
    avg_nfev_dict = {k: v['avg_nfev'] if 'avg_nfev' in v.keys() else v['nfev'] for k,v in stats_dict.items()}
    myplot(avg_nfev_dict, label)

plt.xlabel("# Layers")
plt.ylabel("Function Evals")
plt.title('Average # of Function Evals per layer')
plot_avg_nfev(no_mub_stats, "No MUBs")
plot_avg_nfev(prepend_stats, "Prepended MUB states")
plt.legend()
plt.show()

### Percent of Advantageous Additions

In [ ]:
def plot_adv_percent(stats_dict: Dict[int, Dict[str, any]], label: str):
    adv_percent_dict = {k: v['adv_addition_percent'] for k,v in stats_dict.items()}
    myplot(adv_percent_dict, label)
    
plt.xlabel("# Layers")
plt.ylabel('Fraction of Advantageous Additions')
plt.title('Fraction of Advantageous Additions')
plot_adv_percent(prepend_stats, "Prepended MUB states")
plt.legend()
plt.show()

### Average \# Of Function Evals in Advantageous Additions

In [ ]:
def plot_adv_avg_nfev(stats_dict: Dict[int, Dict[str, any]], label: str):
    adv_percent_dict = {k: v['adv_avg_nfev'] for k,v in stats_dict.items()}
    myplot(adv_percent_dict, label)
    
plt.xlabel("# Layers")
plt.ylabel('Function Evals')
plt.title('Average Function Evals for advantageous additions')
plot_min_nfev(no_mub_stats, "No MUBs (baseline)")
plot_adv_avg_nfev(prepend_stats, "Prepended MUB states")
plt.legend()
plt.show()

### Summary: MUB Prepending

In [ ]:
plot_min_nfev(no_mub_stats, "No MUBs (baseline)")
plot_min_nfev(prepend_stats, 'Minimal Function Evals')
plot_avg_nfev(prepend_stats, 'Average Function Evals')
plot_adv_avg_nfev(prepend_stats, 'Average Adventageous NFEV')
plt.xlabel("# Layers")
plt.ylabel('Function Evals')
plt.title('Function Eval data for Prepended States')
plt.legend()
plt.show()